In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
#get mnist data, with one_hot encoding
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
num_train = mnist.train.num_examples #55,000
num_validation = mnist.validation.num_examples #5000
num_test = mnist.test.num_examples #10,000

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

<h1> Part a </h1>

In [24]:
#learning rate
lr = 0.01
#number of traning steps
num_steps =500
#number of batch_size
batch_size = 128

#network parameters
n_hidden_1 = 300
num_input = 784
num_classes = 10

layers_dims = [num_input, n_hidden_1, num_classes]

In [4]:
def cross_entropy(predictions, targets, epsilon=1e-12):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions. 
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray        
    Returns: scalar
    """
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    N = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions))/N
    return ce

In [34]:
def initialize_parameters(layers_dims):
    np.random.seed(1)               
    parameters = {}
    L = len(layers_dims)            

    for l in range(1, L):           
        parameters["W" + str(l)] = np.random.normal(0,1,(layers_dims[l - 1],layers_dims[l]))
        parameters["b" + str(l)] = np.zeros((1,layers_dims[l]))

        #print(parameters["W" + str(l)].shape)
        assert parameters["W" + str(l)].shape == (
             layers_dims[l - 1],layers_dims[l])
        assert parameters["b" + str(l)].shape == (1,layers_dims[l])
    
    return parameters

In [8]:
def relu(Z):
    A = np.maximum(0, Z)
    return A, Z

def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    return A, Z

In [64]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    #print( np.sum(np.exp(x), axis=1).shape)
    return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

<h1> Part b </h1>

In [44]:
def linear_forward(A_prev, W, b):
    Z = np.dot(A_prev, W) + b
    cache = (A_prev, W, b)
    return Z, cache

In [47]:
def linear_activation_forward(A_prev, W, b, activation_fn):
    assert activation_fn == "sigmoid" or activation_fn == "relu"

    if activation_fn == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)

    elif activation_fn == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)

    assert A.shape == (A_prev.shape[0], W.shape[1])

    cache = (linear_cache, activation_cache)
    return A, cache

In [17]:
input_x, input_y = mnist.train.next_batch(batch_size)
print(input_x.shape)
print(input_y.shape)

(128, 784)
(128, 10)


In [35]:
parameters = initialize_parameters(layers_dims)

In [37]:
#print(parameters['W1'])

[[ 1.62434536 -0.61175641 -0.52817175 ...  0.78477065 -0.95542526
   0.58591043]
 [ 2.06578332 -1.47115693 -0.8301719  ...  0.98635218 -0.46615486
   1.38499134]
 [-1.07296428  0.49515861 -0.9520621  ... -1.45832446  0.49434165
  -1.49419377]
 ...
 [-0.57311171 -0.85178662 -1.92939729 ... -1.6302608   2.30060654
   0.78945511]
 [-0.93341628  0.84227581  1.41383363 ... -0.25037006  0.73473073
   0.29645299]
 [-1.09702598  0.70433309 -0.50254528 ... -0.38962755  0.8069491
  -1.96658544]]


In [51]:
hidden_x, cache_1 = linear_activation_forward(input_x, parameters['W1'], parameters['b1'],"relu")
#print(hidden_x.shape)

(128, 300)


In [52]:
output_x, cache_2 = linear_activation_forward(hidden_x, parameters['W2'], parameters['b2'],"sigmoid")
#print(output_x.shape)

(128, 10)


In [66]:
output_y = softmax(output_x)
#print(output_y[0])

[0.05367289 0.14589805 0.14589805 0.14589805 0.05581817 0.14589805
 0.05367289 0.05367289 0.05367289 0.14589805]


In [68]:
cost = cross_entropy(output_y,input_y)
print('The cost is: ', cost)

The cost is:  2.5071551018139853
